importとか諸々

In [256]:
import pandas as pd
import numpy as  np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

データの読み込み

In [257]:
test = pd.read_csv('test.csv')
year1 = pd.read_csv('year1.csv')
year2 = pd.read_csv('year2.csv')
year3 = pd.read_csv('year2.csv')
year4 = pd.read_csv('year2.csv')

欠損値を0こ以上含むデータの確認

In [258]:
print(test.isnull().sum()[test.isnull().sum()>0].sort_values(ascending=False))
print(year1.isnull().sum()[year1.isnull().sum()>0].sort_values(ascending=False))
print(year2.isnull().sum()[year2.isnull().sum()>0].sort_values(ascending=False))
print(year3.isnull().sum()[year3.isnull().sum()>0].sort_values(ascending=False))
print(year4.isnull().sum()[year4.isnull().sum()>0].sort_values(ascending=False))

Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)


データをまとめる

In [259]:
all_data = pd.concat([year1,year2,year3,year4])

In [260]:
all_data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X57,X58,X59,X60,X61,X62,X63,X64,id,TARGET
0,0.20055,0.37951,0.39641,2.0472,32.351,0.38825,0.24976,1.3305,1.1389,0.50494,...,0.39718,0.87804,0.001924,8.416,5.1372,82.658,4.4158,7.4277,0,0
1,0.20912,0.49988,0.47225,1.9447,14.786,0,0.25834,0.99601,1.6996,0.49788,...,0.42002,0.853,0,4.1486,3.2732,107.35,3.4,60.987,1,0
2,0.24866,0.69592,0.26713,1.5548,-1.1523,0,0.30906,0.43695,1.309,0.30408,...,0.81774,0.76599,0.69484,4.9909,3.951,134.27,2.7185,5.2078,2,0
3,0.081483,0.30734,0.45879,2.4928,51.952,0.14988,0.092704,1.8661,1.0571,0.57353,...,0.14207,0.94598,0,4.5746,3.6147,86.435,4.2228,5.5497,3,0
4,0.18732,0.61323,0.2296,1.4063,-7.3128,0.18732,0.18732,0.6307,1.1559,0.38677,...,0.48431,0.86515,0.12444,6.3985,4.3158,127.21,2.8692,7.898,4,0


含まれているstrの処理

In [278]:
for i in all_data:
  all_data[i] = pd.to_numeric(all_data[i],errors='coerce')
all_data = all_data.dropna()

In [280]:
for i in test:
  test[i] = pd.to_numeric(test[i],errors='coerce')
test = test.dropna()

Xとyの決定

In [283]:
X = all_data.iloc[:, 0:-2]
test_X = test.iloc[:,0:-1]

In [284]:
y = all_data.loc[:, 'TARGET']

データを分割

In [285]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.3,random_state=0)

学習（とりあえずランダムフォレスト）

In [286]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [287]:
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [288]:
rf.score(X_test,y_test)

0.9959034066407935

In [289]:
out_put = rf.predict(test_X)

In [290]:
submission = pd.DataFrame({'id': test['id'], 'TARGET': out_put})
submission.to_csv('submission_second.csv', index=False)

KeyError: 'PassengerId'